In [ ]:
def add_line(file, line):
    file.write(f'{line}\n')

In [ ]:
packages = [
    '\\usepackage[T1]{fontenc}',
    '\\usepackage[utf8]{inputenc}',
    '\\usepackage{lmodern}',
    '\\usepackage{textcomp}',
    '\\usepackage{lastpage}',
    '\\usepackage{graphicx}',
    '\\usepackage{longtable}',
    '\\usepackage[sorting=none]{biblatex}',
    '\\usepackage{booktabs}',
]
title = 'Supplemental Appendix'
bib_filename = 'austcovid'
with open('appendix.tex', 'w') as app_file:
    add_line(app_file, '\\documentclass{article}')
    for line in packages:
        add_line(app_file, line)
    add_line(app_file, f'\\addbibresource{{{bib_filename}.bib}}')
    add_line(app_file, f'\\title{{{title}}}')
    add_line(app_file, '\\begin{document}')
    add_line(app_file, 'test text')
    add_line(app_file, '\\printbibliography')
    add_line(app_file, '\\end{document}')